In [2]:
from typing import Tuple, Optional, cast
import numpy as np
from dataclasses import dataclass
import casadi as ca

In [ ]:
def MX_horzcat(*args: ca.MX) -> ca.MX:
    return ca.horzcat(*args)


def SX_horzcat(*args: ca.SX) -> ca.SX:
    return ca.horzcat(*args)


def DM_horzcat(*args: ca.DM) -> ca.DM:
    return ca.horzcat(*args)


def MX_vertcat(*args: ca.MX) -> ca.MX:
    return ca.vertcat(*args)


def SX_vertcat(*args: ca.SX) -> ca.SX:
    return ca.vertcat(*args)


def DM_vertcat(*args: ca.DM) -> ca.DM:
    return ca.vertcat(*args)


def create_symbolic_scalar(name: str, size: Optional[Tuple[int, int]] = None) -> ca.SX:
    return ca.SX.sym(name, size) if size else ca.SX.sym(name)


def create_symbolic_matrix(name: str, size: Optional[Tuple[int, int]] = None) -> ca.MX:
    return ca.MX.sym(name, size) if size else ca.MX.sym(name)

In [ ]:
@dataclass
class AgentData:
    id: int
    radius: float
    initial_position: Tuple[float, float]
    initial_orientation: float
    planning_time_step: float
    initial_linear_velocity: float
    initial_angular_velocity: float
    planning_horizon: int
    sensor_radius: float
    avoid_obstacles: bool
    linear_velocity_bounds: Tuple[float, float]
    angular_velocity_bounds: Tuple[float, float]
    linear_acceleration_bounds: Tuple[float, float]
    angular_acceleration_bounds: Tuple[float, float]
    goal_position: Tuple[float, float] = None
    goal_orientation: float = None
    use_warm_start: bool = False

    def __post_init__(self):
        self.linear_velocity = self.initial_linear_velocity
        self.angular_velocity = self.initial_angular_velocity
        self.initial_state = np.array([*self.initial_position, self.initial_orientation])
        self.goal_state = np.array([*self.goal_position, self.goal_orientation]) if self.goal_position else self.initial_state
        self.goal_radius = 0.5
        self.states_matrix = np.tile(self.initial_state, (self.planning_horizon + 1, 1)).T
        self.controls_matrix = np.zeros((2, self.planning_horizon))

In [ ]:
@dataclass
class PlannerData:
    planning_time_step: float
    planning_horizon: int

    def __post_init__(self):
        # States
        self.symbolic_states = SX_vertcat(
            create_symbolic_scalar("x"),
            create_symbolic_scalar("y"),
            create_symbolic_scalar("theta"),
        )
        self.num_states = cast(int, self.symbolic_states.numel())

        # Controls
        self.symbolic_controls = SX_vertcat(
            create_symbolic_scalar("a"),
            create_symbolic_scalar("alpha"),
        )
        self.num_controls = cast(int, self.symbolic_controls.numel())

        # Weight matrix for goal cost
        self.weight_matrix = ca.DM(ca.diagcat(100, 100, 0))

        self.angular_acceleration_weight = ca.DM(30)
        self.linear_acceleration_weight = ca.DM(50)

        # Matrix of states over the prediction horizon
        # (contains an extra column for the initial state)
        self.symbolic_states_matrix = create_symbolic_matrix(
            "X", (self.num_states, self.planning_horizon + 1)
        )

        # Matrix of controls over the prediction horizon
        self.symbolic_controls_matrix = create_symbolic_matrix(
            "U", (self.num_controls, self.planning_horizon)
        )

        # Initial state and Goal state vector
        self.symbolic_terminal_states_vector = create_symbolic_matrix(
            "P", (self.num_states + self.num_states, 1)
        )

        # Optimization variables
        self.symbolic_optimization_variables = MX_vertcat(
            self.symbolic_states_matrix.reshape((-1, 1)),
            self.symbolic_controls_matrix.reshape((-1, 1)),
        )

### Model Predictive Control Equations
$$
\begin{aligned}
& \underset{X}{\text{min}}& &f(X; P) = \sum_{t=1}^{N} \omega_x (x_t - x_g)^2 + \omega_y (y_t - y_g)^2 + \omega_\theta (\theta_t - \theta_g)^2 \\ 
& \underset{U}{\text{min}}& &f(U) = \sum_{t=0}^{N-1} \omega_a a_t^2 + \omega_x \alpha_t^2 \\ 
& \text{subject to :}& & x_0 - x_I = 0; \quad \text{and} \quad y_0 - y_I = 0; \quad \text{and} \quad \theta _0 - \theta _I = 0\\
& & & \forall t \in \{1, \dots, N\}, \quad x_t - (x_{t-1} + (v_I + \sum_{k=1}^{t} a_{k-1}T) \cos(\theta_{t-1}) T) = 0\\
& & & \forall t \in \{1, \dots, N\}, \quad y_t - (y_{t-1} + (v_I + \sum_{k=1}^{t} a_{k-1}T) \sin(\theta_{t-1}) T) = 0\\
& & & \forall t \in \{1, \dots, N\}, \quad \theta _{t} - (\theta _{t-1} + (\omega _I + \sum_{k=1}^{t} \alpha _{k-1}T) T) = 0\\
& & & \forall i \in \{0, \dots, O\}, \forall t \in \{1, \dots, N\}, \quad \text{dist}(x_t, o_i) \geq I\\
& & & v_L \leq v_i + \sum_{k=1}^{t} a_{k-1}T \leq v_U\\
& & & \omega _L \leq \omega _i + \sum_{k=1}^{t} \alpha _{k-1}T \leq \omega _U\\
& & & \forall t \in \{0, \dots, N+1\}, \quad l_L \leq x_t \leq l_U\\
& & & \forall t \in \{0, \dots, N\}, \quad u_L \leq a_t \leq u_U \quad \text{and} \quad \alpha _L \leq \alpha _t \leq \alpha _U\\
& \text{where :}& & X = \{x_0, \dots, x_N, \quad y_0, \dots, y_N, \quad \theta _0, \dots, \theta_N\} \\
& & & U = \{a_0, \dots, a_{N-1}, \quad \alpha_0, \dots, \alpha_{N-1}\}\\
& & & P = \{x_I, y_I, \theta _I, x_G, y_G, \theta _G\}
\end{aligned}
$$